### WeRateDogs Data Wrangling

#### Import Libraries & Authenticate API


In [ ]:
#importing tools
import pandas as pd
import numpy as np
import tweepy
import requests
import configparser
import json
import time 
import re
from timeit import default_timer as timer
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import seaborn as sns

In [ ]:

# Load Twitter API credentials securely
API_KEY = "your_api_key_here"
API_SECRET = "your_api_secret_here"
ACCESS_TOKEN = "your_access_token_here"
ACCESS_SECRET = "your_access_secret_here"

# Authenticate with Twitter
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
tweet_ids = archive.tweet_id.values
len(tweet_ids)


2356

#### Load Datasets

In [ ]:
# Function to download and read the image predictions dataset
def download_image_predictions(url, filename):
    """Downloads the image predictions dataset and saves it as a file."""
    response = requests.get(url, allow_redirects=True)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        return pd.read_csv(filename, sep='\t')
    else:
        print("Error downloading file.")
        return None

# Load Twitter archive dataset
archive = pd.read_csv('twitter-archive-enhanced.csv')

# Download image predictions dataset
image_predictions_url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
image_predictions = download_image_predictions(image_predictions_url, 'image-predictions.tsv')


,tweet_id,likes,retweets
0,892420643555336193,39467,8853
1,892177421306343426,33819,6514
2,891815181378084864,25461,4328
3,891689557279858688,42908,8964
4,891327558926688256,41048,9774


#### Fetch Twitter API Data

In [ ]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
tweet_ids = archive["tweet_id"].values
output_file = "tweet_json.txt"
fails_dict = {}

start_time = time.time()
print(f"Fetching {len(tweet_ids)} tweets from Twitter API...\n")

with open(output_file, "w") as outfile:
    for count, tweet_id in enumerate(tweet_ids, start=1):
        try:
            tweet = api.get_status(tweet_id, tweet_mode="extended")
            json.dump(tweet._json, outfile)
            outfile.write("\n")  # Ensure each tweet is stored in a new line
            print(f"✔ {count}: Successfully fetched tweet ID {tweet_id}")
        except tweepy.TweepError as e:
            print(f"✖ {count}: Failed to fetch tweet ID {tweet_id} - {e}")
            fails_dict[tweet_id] = str(e)  # Store the error message for debugging

end_time = time.time()
print("\n--------------------------------------------")
print(f"Completed in {round(end_time - start_time, 2)} seconds.")
print(f"Failed tweets: {len(fails_dict)}")
print("--------------------------------------------\n")

# Save failed tweet IDs for reprocessing
with open("failed_tweets.json", "w") as failed_file:
    json.dump(fails_dict, failed_file, indent=4)

# Quick data overview
def summarize_data(df, name):
    """Prints a quick summary of a dataframe."""
    print(f"\nSummary for {name}:")
    print(df.info())
    print(df.head())
    print("-" * 50)

summarize_data(archive, "Twitter Archive")

summarize_data(image_predictions, "Image Predictions")

# Data Cleaning - Removing Retweets
archive_cleaned = archive[archive.retweeted_status_id.isna()].copy()

# Dropping unnecessary columns
columns_to_drop = [
    'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id',
    'retweeted_status_user_id', 'retweeted_status_timestamp'
]
archive_cleaned.drop(columns=columns_to_drop, inplace=True)

# Converting timestamp to datetime format
archive_cleaned['timestamp'] = pd.to_datetime(archive_cleaned['timestamp'])

# Fixing rating inconsistencies
def extract_ratings(text):
    """Extracts rating numerator and denominator from tweet text."""
    match = re.search(r"(\d+)/(\d+)", text)
    if match:
        return int(match.group(1)), int(match.group(2))
    return None, None

archive_cleaned[['rating_numerator', 'rating_denominator']] = archive_cleaned[
    'text'
].apply(lambda x: pd.Series(extract_ratings(x)))

# Filtering out invalid ratings
archive_cleaned = archive_cleaned[archive_cleaned['rating_denominator'] == 10]

# Merging datasets
final_df = archive_cleaned.merge(image_predictions, on='tweet_id', how='left')

# Saving cleaned dataset
final_df.to_csv("twitter_archive_master.csv", index=False)

#### Data Summarization

In [ ]:
# Quick data overview
def summarize_data(df, name):
    """Prints a quick summary of a dataframe."""
    print(f"Summary for {name}:")
    print(df.info())
    print(df.head())
    print("-" * 50)

summarize_data(archive, "Twitter Archive")
summarize_data(image_predictions, "Image Predictions")


#### Programatic Assessments

In [ ]:
import io
import sys

# Capture and format output
buffer = io.StringIO()
sys.stdout = buffer
archive.info()
sys.stdout = sys.__stdout__

# Print formatted output
print("\n**Programmatic Assessment of `archive` Dataset:**")
print(buffer.getvalue())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [ ]:
# First 5 Rows of Archive Data
display(archive.head())

# Checking Reply Status ID, User ID, and Source
archive_subset = archive[['in_reply_to_status_id', 'in_reply_to_user_id', 'source']]
display(archive_subset.tail())

# Expanded URLs
display(archive['expanded_urls'].head())

# Rating Numerator Counts
display(archive.rating_numerator.value_counts())

# Rating Denominator Counts
display(archive.rating_denominator.value_counts())


In [ ]:
# Checking for Potentially Invalid Dog Names
display(archive[archive.name.str.islower() == True].name.value_counts())


In [ ]:
# Image Predictions Data Overview
print(image_predictions.info())
display(image_predictions.head())
display(image_predictions.describe())

# Checking Prediction Classifications
prediction_img_df = image_predictions[['p1', 'p2', 'p3']]
display(prediction_img_df.head(10))


In [ ]:
# Non-Dog Predictions
num_p1_notdog = image_predictions.p1_dog.value_counts().get(False, 0)
num_p2_notdog = image_predictions.p2_dog.value_counts().get(False, 0)
num_p3_notdog = image_predictions.p3_dog.value_counts().get(False, 0)
print(f" Non-Dog Predictions - P1: {num_p1_notdog}, P2: {num_p2_notdog}, P3: {num_p3_notdog}")


In [ ]:
# Twitter API JSON Data Overview
jsonlist = pd.read_json("tweet_json.txt", lines=True)
print(jsonlist.info())
display(jsonlist.head())


## Data Quality Issues
During the assessment of the dataset, the following quality issues were identified:

### Twitter Archive
1. **Timestamp Formatting**  
   - The `timestamp` and `retweeted_status_timestamp` columns are stored as objects (string format) instead of `datetime`, requiring conversion.

2. **Rating Outliers**  
   - Some ratings have denominators greater than 10. While higher-than-10 ratings are a humorous feature of WeRateDogs, some extreme outliers exist and require review.

3. **Invalid Dog Names**  
   - Several names are incorrect or placeholders (e.g., `"a"`, `"the"`, `"an"`), likely due to parsing errors.

4. **Columns with Mostly Null Values**  
   - The following columns contain mostly missing data and may not be useful:  
     - `in_reply_to_status_id` (78 non-null)  
     - `in_reply_to_user_id` (78 non-null)  
     - `retweeted_status_id` (181 non-null)  
     - `retweeted_status_user_id` (181 non-null)  
     - `retweeted_status_timestamp` (181 non-null)  

5. **Source Column Contains HTML**  
   - The `source` column includes raw HTML tags, which should be extracted for readability.

6. **Missing URLs**  
   - There are only 2,297 URLs, but the dataset contains 2,356 total entries, suggesting that some URLs are missing.

### Image Predictions File
7. **Missing Predictions**  
   - The Twitter archive has 2,356 tweets, but the image predictions dataset contains only 2,075 entries, meaning some tweets lack image-based classification.

8. **Tweet ID Data Type**  
   - The `tweet_id` column should be stored as a string, not an integer, to prevent unintended conversions and maintain consistency.

### API Data File
9. **Missing API Data Entries**  
   - The API data contains 2,354 entries, meaning two tweets are missing.

10. **Tweet ID Data Type**  
   - Similar to the image predictions file, the `tweet_id` column should be stored as a string instead of an integer.


## Tidiness Issues
1. **Dog Stages Should Be Combined**  
   - The `doggo`, `floofer`, `pupper`, and `puppo` columns should be consolidated into a single column representing dog stages.

2. **Files Should Be Merged**  
   - The Twitter archive, image predictions, and API data files should be combined into a single, unified dataframe.


## Cleaning Data
In this section, all documented **quality and tidiness issues** will be addressed.

In [ ]:
# Create copies of the original datasets to preserve raw data
df_archive = archive.copy()
df_predict = image_predictions.copy()
df_api = jsonlist.copy()

# Display first row of the Twitter archive dataset
df_archive.head(1) 

### Changing datatypes

##### Define: Some columns have datatypes that are incorrect. 
Convert `tweet_id` to **string** in all datasets.
Convert `timestamp` to **datetime**.
Convert `rating_numerator` to **float**.

In [ ]:
# Convert tweet_id to string
df_archive['tweet_id'] = df_archive['tweet_id'].astype(str)
df_predict['tweet_id'] = df_predict['tweet_id'].astype(str)
df_api['tweet_id'] = df_api['tweet_id'].astype(str)

In [ ]:
# Convert timestamp to datetime
df_archive['timestamp'] = pd.to_datetime(df_archive['timestamp'])

# Convert rating_numerator to float
df_archive['rating_numerator'] = df_archive['rating_numerator'].asty

In [ ]:
# Verify data type changes
df_archive.info()
df_predict.info()
df_api.info()

### **Handling Rating Denominator Inconsistencies**

>The WeRateDogs scoring system is a N/10 system with N being a number that is often greater than 10 due to the nature of the humorous scoring system. 

>Some denominator values exceet 10, which is not expected.

>Normalize ratings by adjusting values to fit the standard scale.

In [ ]:
# Identify tweets where denominator is not 10
discrepant_ratings = df_archive[df_archive['rating_denominator'] != 10]
discrepant_ratings

# Normalize ratings to a /10 scale
df_archive['rating_numerator'] = (df_archive['rating_numerator'] / df_archive['rating_denominator']) * 10
df_archive['rating_denominator'] = 10

# Verify changes
df_archive[['tweet_id', 'rating_numerator', 'rating_denominator']].head(10)

### **Merging Datasets and Storing Data**

> Merge `df_archive`, `df_predict`, and `df_api` on `tweet_id`.

> Use **left join** to preserve all tweet archive data.


In [ ]:
# Merge datasets on tweet_id
final_df = df_archive.merge(df_predict, on='tweet_id', how='left')
final_df = final_df.merge(df_api, on='tweet_id', how='left')

# Save cleaned dataset
final_df.to_csv("twitter_archive_master.csv", index=False)

# Verify merged dataset structure
final_df.info()

### **Cleaning HTML from Source Column**

> The `source` column contains **HTML anchor tags** that should be extracted.


In [ ]:
def extract_source(html_string):
    """Extracts text from an HTML anchor tag."""
    match = re.search(r">(.*?)<", html_string)
    return match.group(1) if match else html_string

# Apply function to source column
df_archive['source'] = df_archive['source'].apply(extract_source)

# Verify changes
df_archive['source'].head()

### **Handling Missing Data**

> Drop columns with **mostly missing values**.

> Ensure URLs are properly filled.

In [ ]:

# Drop columns with excessive missing values
columns_to_drop = [
    'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id',
    'retweeted_status_user_id', 'retweeted_status_timestamp'
]
df_archive.drop(columns=columns_to_drop, inplace=True)

# Fill missing URLs with 'No URL Available'
df_archive['expanded_urls'].fillna("No URL Available", inplace=True)

# Verify missing data is handled
df_archive.info()

### **Final Dataset Verification**

In [ ]:

# Display final cleaned dataset structure and first few rows
final_df.info()
final_df.head()

### **Summary of Cleaning Steps**

-Converted data types where needed.

-Handled rating inconsistencies by normalizing to `/10` scale.

-**Merged datasets** to create a final master DataFrame.

-Extracted clean text from HTML in the `source` column.

-Removed highly null columns and ensured URLs are complete.

-Saved the cleaned dataset for further analysis.





## Analyzing and Visualizing Data

In [ ]:
#Summary Statistics of the Cleaned Dataset
final_df.describe()


In [ ]:
# Most Common Dog Names
final_df['name'].value_counts().head(5)


In [ ]:
# Most Common Tweet Sources
final_df['source'].value_counts()

In [ ]:
# Calculate Percentage of Non-Dog Predictions for p1, p2, and p3
notdog1 = final_df.p1_dog.value_counts(normalize=True).mul(100)[False]
notdog2 = final_df.p2_dog.value_counts(normalize=True).mul(100)[False]
notdog3 = final_df.p3_dog.value_counts(normalize=True).mul(100)[False]

# Store results in a list
notdogvals = [notdog1, notdog2, notdog3]
notdogvals

In [ ]:
# Most Retweeted Dog Names
final_df.groupby('name')['retweets'].sum().sort_values(ascending=False)

# Most Retweeted Dog Labels
final_df.groupby('label')['retweets'].sum().sort_values(ascending=False)

In [ ]:
# Modify Non-Name Values for Cleaner Visualizations
final_df['name'] = final_df['name'].apply(lambda x: 'Unknown' if x.lower() in ['a', 'the', 'an', 'by'] else x)

### **Summary of Findings**
- **Charlie** is the most common dog name.
- **iPhone is the most used platform** for tweeting.
- **A high percentage of third predictions are non-dogs.**
- **Bo, Stephan, and Duddles are the most retweeted dog names.**
- **Pupper is the most abundant label.**

Further visualizations will be conducted to support these findings.

## Visualizations

The most interesting insights involve the number of retweets for different dog names and labels. The following visualizations highlight these trends.


#### Retweeted Dog Names

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Aggregate total retweets by dog name, excluding 'None' and 'No Name'
name_retweets = final_df.groupby('name')['retweets'].sum().sort_values(ascending=False).nlargest(7)
name_retweets = name_retweets.drop(index=['None', 'No Name'], errors='ignore')

# Plot the most retweeted dog names
plt.figure(figsize=(10, 5))
sns.barplot(x=name_retweets.index, y=name_retweets.values, palette='viridis')
plt.title("Most Retweeted Dog Names")
plt.xlabel("Dog Names")
plt.ylabel("Number of Retweets")
plt.xticks(rotation=45)
plt.savefig("most_retweeted_dog_names.png")

##### Retweeted Dog Labels

In [ ]:
# Aggregate total retweets by dog label
dog_label = final_df.groupby('label')['retweets'].sum().sort_values(ascending=False)

# Plot the most retweeted dog labels
plt.figure(figsize=(10, 5))
sns.barplot(x=dog_label.index[:10], y=dog_label.values[:10], palette='magma')
plt.title("Most Retweeted Dog Labels")
plt.xlabel("Dog Labels")
plt.ylabel("Number of Retweets")
plt.xticks(rotation=45)
plt.savefig("most_retweeted_dog_labels.png")

These visualizations provide a clearer understanding of the engagement levels associated with different dog names and classifications.